# Using Python with HST Spectral Data

## 1. Retrieving Data from MAST
***First, make a directory in your central store to put your data in.***

```cd /user/myname
mkdir spect_training```

`cd spect_training`

`mkdir mast_data`

`cp /user/jotaylor/new_hire_training/*fits .`

For HST spectral data, there are three types of files:
* raw 
    * COS: `rawtag`, `rawaccum`, `rawacq`
    * STIS: `raw`, `tag`, `wav`
* support
    * `asn`, `spt`, `trl`, and more
* intermediate
    * `corrtag`, `crj`, `flt`, and more
* products
    * `x1d`, `x1dsum`

Raw data has undergone no processing besides generic conversion from spacecraft data to FITS. Intermediate products are files produced by each instrument’s calibration pipeline on the way to extracting a spectrum.  Products are fully calibrated, extracted 1-D spectra and typically have `x1d` in the extension. See the Instrument Data Handbooks for a full list of all data file types.

To retrieve STIS data by its filename, simply enter the name in "Dataset" field in a [MAST search](http://archive.stsci.edu/hst/search.php). For COS data, however, you must search by the individual dataset's association name (more info [here](http://www.stsci.edu/hst/cos/documents/handbooks/datahandbook/ch2_cos_data5.html#460268)).

***Retrieve two datasets, lbgu17qnq (association lbgu17010) and o8k401010 from the archive.*** Be sure to retrieve both un-calibrated and calibrated products.


`ls`

First, let's look at the primary headers and identify some important parameters.

In [ ]:
# Run this cell.
from astropy.io import fits
cos_raw_a = "lbgu17qnq_rawtag_a.fits"
cos_raw_b = "lbgu17qnq_rawtag_b.fits"
stis_raw = "o8k401010_raw.fits" 
cos_hdr0 = fits.getheader(cos_raw_a, 0)
cos_hdr0

***Print the STIS header as well.***

In [ ]:
# your code here

You can also get individual header keywords. These are not case-specific in astropy.

In [ ]:
# Run this cell.
print("DETECTOR: ", cos_hdr0["DETECTOR"])
print("detector: ", cos_hdr0["detector"])

Let's get some useful information about the datasets. ***Use a loop like the one below, but add any other keywords you think should be included. Make sure the keywords are identical between instruments.***

In [ ]:
for hdr in [cos_hdr0, stis_hdr0]:
    try:
        print(hdr["instrument"])
    except KeyError:
        print(hdr["primesi"])
    for key in ["detector", "obsmode", "opt_elem", "targname"]:
        print("{0}: {1}".format(key, hdr[key]))
    print()    

What is the difference between ACCUM and TIME-TAG? Let's look at the structure of each raw dataset to find out.

In [ ]:
# Run this cell.
print(fits.info(cos_raw_a), "\n")
print(fits.info(stis_raw))

You can get pertinent information of a FITS Table by looking at the column definitions.

In [ ]:
# Run this cell.
with fits.open(cos_raw_a) as hdulist:
    data1 = hdulist[1].data
print(data1.names)
print(data1.columns)
print(data1.columns[0])
print(data1.columns[0].unit)

## 2. Calibration
There is a calibration pipeline for each instrument. For COS and STIS, they are CalCOS and CalSTIS. The pipelines themselves have very limited optional arguments, and very few will effect the data. Most of the ways to change how data is calibrated is accessed by editing header keywords in the raw files. 

All of the reference files used by the calibration pipelines are located in central storage, and the pipelines themselves look to environment variables to find them. If you have not already done so, you will need to set these environment variables now.

In [ ]:
# Run this cell.
import os
os.environ["oref"] = "/grp/hst/cdbs/oref/"
os.environ["lref"] = "/grp/hst/cdbs/lref/"

In [ ]:
# Run this cell.
from stistools import calstis
import calcos
print("CalCOS version: {0}".format(calcos.__version__))
print("CalCOS vdate: {0}".format(calcos.__vdate__))
print("CalSTIS version: {0}".format(calstis.__version__))
print("CalSTIS vdate: {0}".format(calstis.__vdate__))

In [ ]:
# Run this cell.
calstis.calstis?

In [ ]:
# Run this cell.
calcos.calcos?

You have already received calibrated products from the archive, but sometimes you may decide to turn certain calibration switches on or off to modify data. Let's try turning off background subtraction switch, `BACKCORR`, then re-calibrating the data. It's a good idea to print to the screen/capture the trailer file that a pipeline creates- it shows step-by-step records of how the data were calibrated and will record any errors that occur. You will need to specify a different output directory to make sure that the pipelines will not overwrite the data you already have.

In [ ]:
# Run this cell.
for cosfile in [cos_raw_a, cos_raw_b]:
    print("Before, {0} BACKCORR={1}".format(cosfile, fits.getval(cosfile,"BACKCORR")))
    with fits.open(cosfile, mode="update") as hdulist:
        hdr0 = hdulist[0].header
        hdr0.set("BACKCORR", "OMIT")
    print("After, {0} BACKCORR={1}".format(cosfile, fits.getval(cosfile,"BACKCORR")))

`mkdir cos_backcorr_omit`

In [ ]:
# Run this cell.
# calcos.calcos(cos_asn, outdir="cos_backcorr_omit")
cp /user/jotaylor/new_hire_training/cos_backcorr_omit/* /user/myname/spect_training/cos_backcorr_omit

***Change the BACKCORR switch for the STIS data and calibrate the data.  You mut first create the output directory for CalSTIS.***

In [ ]:
# your code here to change BACKCORR

`mkdir stis_backcorr_omit`

In [ ]:
# Calibrate the STIS raw file, and be sure to specify a trailing slash for the output directory.
calstis.calstis(stis_raw, outroot="stis_backcorr_omit/")

`corrtag` files are very useful, but they are only produced for COS data. Instead, let's look at the unmodified `x1d` products for both COS and STIS. ***Explore the 1st extension data for the original COS and STIS `x1d` files, looking at the columns and names as we did before.***

In [ ]:
# Run this cell.
orig_cos_x1dfile = "lbgu17qnq_x1d.fits"
orig_stis_x1dfile = "o8k401010_x1d.fits"
cos_x1d_orig = fits.getdata(orig_cos_x1dfile, 1)
stis_x1d_orig = fits.getdata(orig_stis_x1dfile, 1)

In [ ]:
# your code here

`x1d` files can have strange formats. Let's fix that...

In [ ]:
# Run this cell.
print(cos_x1d_orig["wavelength"].shape)
print(cos_x1d_orig["segment"])
print(stis_x1d_orig["wavelength"].shape)

In [ ]:
# Run this cell.
import numpy as np
cos_wl_orig = np.concatenate((cos_x1d_orig["wavelength"][1], cos_x1d_orig["wavelength"][0]))
cos_flux_orig = np.concatenate((cos_x1d_orig["flux"][1], cos_x1d_orig["flux"][0]))
stis_wl_orig = stis_x1d_orig["wavelength"].flatten()
stis_flux_orig = stis_x1d_orig["flux"].flatten()
print(cos_wl_orig.shape)
print(stis_wl_orig.shape)

***Read in the BACKCORR=OMIT COS and STIS data and assign the wavelength and flux to arrays.***

In [ ]:
# your code here

Now let's compare the COS data that has been calibrated with `BACKCORR=PERFORM` to `BACKCORR=OMIT`.

In [ ]:
# Run this cell.
%matplotlib inline
import matplotlib
from matplotlib import pyplot as plt
matplotlib.rcParams['figure.figsize'] = (20,7)
plt.style.use("bmh")

In [ ]:
# Run this cell.
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, sharex=True)
ax1.plot(cos_wl_new, cos_flux_new, color="lightseagreen", label="WAVECORR=OMIT")
ax1.plot(cos_wl_orig, cos_flux_orig, color="royalblue", label="WAVECORR=PERFORM")
ax1.legend(loc="best")
ax2.plot(cos_wl_new, cos_flux_orig-cos_flux_new, color="royalblue")
ax2.set_xlabel("Wavelength [$\AA$]")
ax1.set_ylabel("Flux [ergs/s/cm$^2$/$\AA$]")
ax2.set_ylabel("Flux [ergs/s/cm$^2$/$\AA$]")
ax1.set_title("COS {0} BACKCORR=PERFORM".format(cos_hdr0["targname"]))
ax2.set_title("BACKCORR=PERFORM - BACKCORR=OMIT")

***Now make a similar plot for STIS.***

In [ ]:
# your code here

## Interpolating Data
***Make a very simple plot with both the COS and STIS spectra.***

In [ ]:
# Use plt.plot instead of plt.subplots
plt.plot(wl, flux, color=...)

Directly comparing two spectra in wavelength space cannot be done quantitatively unless both datasets have the same wavelength scale. Interpolating one spectrum onto another's wavelength scale will allow you to do far more useful calculations such as dividing, subtracting, etc. ***Follow the example below to perform a linear interpolation of the STIS x1d file onto the wavelength scale of the COS x1d (use the original files, not the BACKCORR=OMIT files).***

In [ ]:
#import numpy as np
#flux2_interpolated = np.interp(wavel1, wavel2, flux2)

In [ ]:
# your code here

***Now divide the two spectra by each other to see where the two differ. Make a plot of the product.***

In [ ]:
# your code here

## Smoothing Data
The COS data we have is at a much higher resolution than that of STIS. ***Smooth the original COS data with a boxcar filter. Try 3 different box sizes: 6, 11, and 21. Make a plot of the original and all smoothed spectra. (Hint: try a loop) (Double hint: try xlim(1220, 1245) and ylim(-0.1e-12, 0.7e-12)***

In [ ]:
#from stsci.convolve import boxcar
#smoothed_flux = boxcar(flux, (boxcar_size,))

In [ ]:
# your code here

## Computing SNR
The signal-to-noise ratio, SNR, is an important statistic of spectra. Signal can be approximated with a low-order polynomial fit to a continuum region where there are no absorption or emission features. Noise can be approximated as the standard deviation of the fit subtracted from the data. ***Smooth the COS data using boxcar=6, and STIS data using boxcar=2. Then, follow the example below to determine the average SNR for the COS and STIS data.***

In [ ]:
#poly2 = np.polyfit(wavelength_region, smoothed_flux_region, 2)
#fit2 = np.poly1d(poly2)
#ycalc = fit2(wavelength_region)
#noise = np.std(smoothed_flux_region - ycalc)
#signal = np.average(ycalc)
#snr = signal / noise

In [ ]:
# your code here